# IRISデータでATDの練習

## こちらで紹介されてたのでやってみる

[Toying with Topological Data Analysis - Part 1 (IRIS)](https://dreamtolearn.com/ryan/data_analytics_viz/47)

ソースはそのサイトの中からリンクされている。これを少しずつ追いかける。

phom というパッケージが必要だが、R のパッケージから外れているので、
[CRAN phom](https://cran.r-project.org/web/packages/phom/index.html)

アーカイブをダウンロードして、Rにインストールしておく。

In [1]:
# まず準備
############################################
## Toying with Topological Data Analysis  ##
## Testing on IRIS Mythica - 200 samples  ##
## four types (3 standard + 1 fabricated) ##
## R. Anderson March 2014                 ##
############################################
library(diffusionMap)
library(randomForest)
library(ggplot2)
library(reshape2)  ## here is where we do a 'pivot table'
library(plyr)
library(phom)

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.
Loading required package: Rcpp


## とりあえずここから
データ読み込みと、散布図での表示。データには mythica 種が入って４種類になっているデータを利用している。
このCSVは適当に入手して、どこかに置いておく。

In [2]:
setwd("~/knime-workspace/dreamtolearn TDA")
iris = read.csv("iris_Mythica.csv") 
## this iris set changed the species to 1,2,3,4 
## or first letter) - doesn not handle strings eg "setosa" 
##  you can also use the original iris 150 set if you change Species to a number or letter
head(iris) # s=setosa; v=virginica; e=versicolor; m=mythica
summary(iris)

,Index,sepal.length,sepal.width,petal.length,petal.width,class
1,1,5.1,3.5,1.4,0.2,setosa
2,2,4.9,3,1.4,0.2,setosa
3,3,4.7,3.2,1.3,0.2,setosa
4,4,4.6,3.1,1.5,0.2,setosa
5,5,5,3.6,1.4,0.2,setosa
6,6,5.4,3.9,1.7,0.4,setosa


     Index         sepal.length    sepal.width     petal.length  
 Min.   :  1.00   Min.   :4.300   Min.   :2.000   Min.   :1.000  
 1st Qu.: 50.75   1st Qu.:5.400   1st Qu.:2.900   1st Qu.:1.700  
 Median :100.50   Median :6.100   Median :3.100   Median :3.395  
 Mean   :100.50   Mean   :5.987   Mean   :3.154   Mean   :3.427  
 3rd Qu.:150.25   3rd Qu.:6.500   3rd Qu.:3.433   3rd Qu.:4.900  
 Max.   :200.00   Max.   :7.900   Max.   :4.400   Max.   :6.900  
  petal.width            class   
 Min.   :0.1000   mythica   :50  
 1st Qu.:0.5175   setosa    :50  
 Median :0.9850   versicolor:50  
 Mean   :1.0783   virginica :50  
 3rd Qu.:1.6000                  
 Max.   :2.5000                  

In [ ]:
# 散布図
## what we are starting with
plot(iris[2:6], main="IRIS Mythica", pch=23, bg = c("red","orange","green", "lightblue")
     [unclass(iris$class)])

## ざっくり diffusion map で見てみる

diffusion map は、次元削減をしてくれる。

In [ ]:
data <- iris[2:5] ## let's learn off 2,3,4,5 columns, first column is index
species <- iris$class ## and species is our target and classifier 

D = dist(scale(data)) # use Euclidean distance on data
## DIST: This function computes and returns the distance matrix computed by using the specified distance measure to compute the distances between the rows of a data matrix.

dmap = diffuse(D,eps.val=250, t=1, neigen=2)
## diffuse: Uses the pair-wise distance matrix for a data set to compute the diffusion map coefficients. Computes the Markov transition probability matrix, and its eigenvalues and left & right eigenvectors. Returns a 'dmap' object.
#eps = 250 and t=1 and neigen = 2 is nice

## PLOT 2
## if this works, you should see clouds of S,M,V and i (or 1,2,3,4) - whatever you had for species tag
plot(dmap$X[,1],dmap$X[,2],col=species,pch=paste(species), 
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2",
     main="Diffusion Map of IRIS Mythica")

ちょっと DIFFUSE の設定を変えてみる。

In [ ]:
# edit DIFFUSE variable values - what changes? 
dmap = diffuse(D,eps.val=3, t=2, neigen=3) 
plot(dmap$X[,1],dmap$X[,2],col=species,pch=paste(species), 
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2")

## 分類一回目
この状態で、RandomForest をやってみる。

In [ ]:
# 2) OK - now Use random forest "Department" classifier to define distances

fit = randomForest(data, species, ntree=100, proximity=TRUE) 
print(fit)
varImpPlot(fit)

random forest のダイアグラム

In [ ]:
# PLOT 3
#version
D2 = 1-fit$proximity # use 1 - proximity
dmap2 = diffuse(D2,eps.val=.5, t=1, neigen=2)   #original dmap1 = diffuse(D1,eps.val=.1, t=1, neigen=2)
head(dmap2)

cluster2 = hclust(dist(dmap2$X[,1:2]))
plot(cluster2); abline(h=.6, col='blue',lwd=3)

In [ ]:
## PLOT 5

plot(dmap2$X[,1],dmap2$X[,2],col=species,pch=paste(species), 
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2")

In [ ]:
## clustering variable 4 (Plot 7) vs 10 is interesting
## depends what you want to output
clustering2 = cutree(cluster2,k=10)  ## this is how many nodes there are
#clustering2

plot(dmap2$X[,1],dmap2$X[,2],col=clustering2, pch=19,
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2")

いったんセーブ。

In [ ]:
## See plot 8 - note number of 'buckets' 

output2 = data.frame(dmap2$X,species,clustering2)
#output2

write.csv(output2,"TDA_export_data2.csv") 

仕切りなおすみたい。

In [ ]:
iris3 = read.csv("iris_Mythica.csv") 
data3 <- iris[2:5] ## let's learn off 4 columns skip 1st
species3 <- iris$class ## and species is our target

random forest やってみる。

In [ ]:
fit3 = randomForest(data3, species3, ntree=100, proximity=TRUE) 

In [ ]:
D3 = 1./fit3$proximity - 1. # use 1/proximity - 1
dmap3 = diffuse(D3,eps.val=80, t=1, neigen=2) 
plot(dmap3$X[,1],dmap3$X[,2],col=species3,pch=paste(species3), 
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2")
## using RF method we generate  plot similar to plot 6 (but better focus)

In [ ]:
cluster3 = hclust(dist(dmap3$X[,1:2]))
plot(cluster3); abline(h=.6, col='green',lwd=3)
clustering3 = cutree(cluster3,k=8)  ## this is how many nodes there are
#clustering3

plot(dmap3$X[,1],dmap3$X[,2],col=clustering3, pch=19,
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2")
## Plot 8
#### Nice - this looks like somehting we can stitch together and ascribe weight to - 

In [ ]:
# このあたりはよくわからないので、実行しなくていい
data <- melt(output3, measure = "clustering3", id = c("species3","clustering3"))
data

print(cast(data, species3 ~ ., sum)) ## useful to show pivot, but not valid add
gold <- print(cast(data, clustering3 ~ ., sum )) ## add up ( so need to divide by bin size)
colnames(gold)[1] <- "cluster"
colnames(gold)[2] <- "weight"
gold[2] = gold[2]/gold[1]  ## need to do this because our pivot gave us SUM, not count (i dont know how to do that)
gold ## ok - we have cluster list, and count (size of ball)
gold[3] = gold[1]+1
colnames(gold)[3] <- "join.to"
gold
### the 8 lines above are probably not the best way to distil information, but best available (room for improvement)
## future: consider a function to modify then export to JSON for D3 consumtion

## ok, let's export this bad boy and see what it looks like in D3 or Google Fusion
write.csv(gold,"TDA_export_data3.csv") 

In [ ]:
# 4) using smoothed histograms
D4 = dist(data)
# dmap4 = diffuse(D4,eps.val=50, t=1, neigen=2)  # works pretty good with iris mythica
dmap4 = diffuse(D4,eps.val=50, t=1, neigen=2) 

plot(dmap4$X[,1],dmap4$X[,2],col=species,pch=paste(species), 
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2", ylim=c(-0.15,0.15))

In [ ]:
# fit RF on smoothed histograms
fit4 = randomForest(data, species, ntree=1000, proximity=TRUE)
print(fit4)

D4 = 1-fit$proximity # use 1 - proximity
dmap4 = diffuse(D4,eps.val=0.3, t=1, neigen=2) 
plot(dmap4$X[,1],dmap4$X[,2],col=species,pch=paste(species), 
     xlab="Diffusion Map Coordinate 1", 
     ylab="Diffusion Map Coordinate 2")
## see plot 6

## ここから、pHom

In [8]:
########## some pHO for good measure
### Source http://blog.revolutionanalytics.com/2014/01/topological-data-analysis-with-r.html 
## (5) Betti where are you? - See Plot 4
data5 <- as.matrix(iris[2:5])
dim(data5)
head(data5)

[1] 200   4

sepal.length,sepal.width,petal.length,petal.width
5.1,3.5,1.4,0.2
4.9,3.0,1.4,0.2
4.7,3.2,1.3,0.2
4.6,3.1,1.5,0.2
5.0,3.6,1.4,0.2
5.4,3.9,1.7,0.4


In [ ]:
max_dim <- 4
max_f <- 1
irisInt0 <- pHom(data5, 
                 dimension=max_dim,              # maximum dimension of persistent homology computed
                 max_filtration_value=max_f,     # maximum dimension of filtration complex
                 mode="vr",                      # type of filtration complex
                 metric="euclidean")

plotBarcodeDiagram(irisInt0, max_dim, max_f, title="H0 Barcode plot of Iris MYTHICA Data")
# Plot 4


In [ ]:
plotPersistenceDiagram(irisInt0, 3, max_f, title="H0 persistence diagram of Iris MYTHICA data")